In [1]:
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from data_processing import LoadData
from model import *
import syft as sy
import copy
hook = sy.TorchHook(torch)
import pandas as pd
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was 'C:\Users\Yash\AppData\Roaming\Python\Python36\site-packages\tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.3.so'


In [2]:
def main():
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"
    
    dataset = pd.read_csv('PeMS_04/PeMS04.csv')
    dataset_size = len(dataset)

    indices = list(range(dataset_size))
    np.random.shuffle(indices)

    split1 = int(np.floor(0.12 * dataset_size)) 
    split2 = int(np.floor(0.24 * dataset_size)) 
    split3 = int(np.floor(0.36 * dataset_size)) 
    split4 = int(np.floor(0.48 * dataset_size)) 
    split5 = int(np.floor(0.6 * dataset_size)) 
    split6 = int(np.floor(0.72 * dataset_size)) 
    split7 = int(np.floor(0.84 * dataset_size)) 
    bobs_indices, alices_indices, harrys_indices, jacks_indices, miks_indices, olivers_indices, kats_indices, elizas_indices = indices[:split1], indices[split1:split2], indices[split2:split3], indices[split3:split4], indices[split4:split5], indices[split5:split6], indices[split6:split7], indices[split7:]

    bobs_sampler = SubsetRandomSampler(bobs_indices)
    alices_sampler = SubsetRandomSampler(alices_indices)
    harrys_sampler = SubsetRandomSampler(harrys_indices)
    jacks_sampler = SubsetRandomSampler(jacks_indices)
    miks_sampler = SubsetRandomSampler(miks_indices)
    olivers_sampler = SubsetRandomSampler(olivers_indices)
    kats_sampler = SubsetRandomSampler(kats_indices)
    elizas_sampler = SubsetRandomSampler(elizas_indices)
        
    train_data = LoadData(data_path=["PeMS_04/PeMS04.csv", "PeMS_04/PeMS04.npz"], num_nodes=307, divide_days=[45, 14],
                              time_interval=5, history_length=6,
                              train_mode="train")


    bobs_loader = DataLoader(train_data, batch_size=64, num_workers=32, sampler=bobs_sampler)
    alices_loader = DataLoader(train_data, batch_size=64, num_workers=32, sampler=alices_sampler)
    harrys_loader = DataLoader(train_data, batch_size=64, num_workers=32, sampler=harrys_sampler)
    jacks_loader = DataLoader(train_data, batch_size=64, num_workers=32, sampler=jacks_sampler)
    miks_loader = DataLoader(train_data, batch_size=64, num_workers=32, sampler=miks_sampler)
    olivers_loader = DataLoader(train_data, batch_size=64, num_workers=32, sampler=olivers_sampler)
    kats_loader = DataLoader(train_data, batch_size=64, num_workers=32, sampler=kats_sampler)
    elizas_loader = DataLoader(train_data, batch_size=64, num_workers=32, sampler=elizas_sampler)
    
#     bob alice harry jack    
#     # Loading Dataset
#     train_data = LoadData(data_path=["PeMS_04/PeMS04.csv", "PeMS_04/PeMS04.npz"], num_nodes=307, divide_days=[45, 14],
#                           time_interval=5, history_length=6,
#                           train_mode="train")

#     train_loader = DataLoader(train_data, batch_size=64, shuffle=True, num_workers=32)
#....................................................
#     test_data = LoadData(data_path=["PeMS_04/PeMS04.csv", "PeMS_04/PeMS04.npz"], num_nodes=307, divide_days=[45, 14],
#                          time_interval=5, history_length=6,
#                          train_mode="test")

#     test_loader = DataLoader(test_data, batch_size=64, shuffle=True, num_workers=32)
    
#     for data in train_loader: 
#         print(data)

    #model = GCN(in_c=6 , hid_c=6 ,out_c=1)
    #ChebNet(in_c=6, hid_c=32, out_c=1, K=2)
    #GATNet(in_c=6 , hid_c=6 ,out_c=1, n_heads=1)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    bobs_model = GATNet(in_c=6 , hid_c=6 ,out_c=1, n_heads=1)
    bobs_model = bobs_model.to(device)
    
    alices_model = GATNet(in_c=6 , hid_c=6 ,out_c=1, n_heads=1)
    alices_model = alices_model.to(device)
    
    harrys_model = GATNet(in_c=6 , hid_c=6 ,out_c=1, n_heads=1)
    harrys_model = harrys_model.to(device)
    
    jacks_model = GATNet(in_c=6 , hid_c=6 ,out_c=1, n_heads=1)
    jacks_model = jacks_model.to(device)
    
    miks_model = GATNet(in_c=6 , hid_c=6 ,out_c=1, n_heads=1)
    miks_model = miks_model.to(device)
    
    olivers_model = GATNet(in_c=6 , hid_c=6 ,out_c=1, n_heads=1)
    olivers_model = olivers_model.to(device)
    
    kats_model = GATNet(in_c=6 , hid_c=6 ,out_c=1, n_heads=1)
    kats_model = kats_model.to(device)
    
    elizas_model = GATNet(in_c=6 , hid_c=6 ,out_c=1, n_heads=1)
    elizas_model = elizas_model.to(device)
    
    #new
    bob = sy.VirtualWorker(hook, id="bob")
    alice = sy.VirtualWorker(hook, id="alice")
    harry = sy.VirtualWorker(hook, id="harry")
    jack = sy.VirtualWorker(hook, id="jack")
    mik = sy.VirtualWorker(hook, id="mik")
    oliver = sy.VirtualWorker(hook, id="oliver")
    kat = sy.VirtualWorker(hook, id="kat")
    eliza = sy.VirtualWorker(hook, id="eliza")
    secure_worker = sy.VirtualWorker(hook, id="secure_worker")

    #new
#     bobs_model = model.copy().send(bob)
#     alices_model = model.copy().send(alice)

    criterion = nn.MSELoss()
    
    #new
    bobs_opt = optim.Adam(params=bobs_model.parameters())
    alices_opt = optim.Adam(params=alices_model.parameters())
    harrys_opt = optim.Adam(params=harrys_model.parameters())
    jacks_opt = optim.Adam(params=jacks_model.parameters())
    miks_opt = optim.Adam(params = miks_model.parameters())
    olivers_opt = optim.Adam(params = olivers_model.parameters())
    kats_opt = optim.Adam(params = kats_model.parameters())
    elizas_opt = optim.Adam(params = elizas_model.parameters())
    
    
    # Train model
    Epoch = 8

    bobs_model.train()
    alices_model.train()
    harrys_model.train()
    jacks_model.train()
    miks_model.train()
    olivers_model.train()
    kats_model.train()
    elizas_model.train()
    
    for epoch in range(Epoch):
    	#new
        epoch_loss_bob = 0.0
        epoch_loss_alice = 0.0
        epoch_loss_harry = 0.0
        epoch_loss_jack = 0.0
        epoch_loss_mik = 0.0
        epoch_loss_oliver = 0.0
        epoch_loss_kat = 0.0
        epoch_loss_eliza = 0.0

        start_time = time.time()
        for data in bobs_loader:  # ["graph": [B, N, N] , "flow_x": [B, N, H, D], "flow_y": [B, N, 1, D]]
            bobs_model.zero_grad()
            predict_value = bobs_model(data, device).to(torch.device("cpu"))  # [0, 1] -> recover
            loss = criterion(predict_value, data["flow_y"])
            epoch_loss_bob += loss.item()
            loss.backward()
            bobs_opt.step()
            
        for data in alices_loader:
            alices_model.zero_grad()
            predict_value = alices_model(data, device).to(torch.device("cpu"))  # [0, 1] -> recover
            loss = criterion(predict_value, data["flow_y"])
            epoch_loss_alice += loss.item()
            loss.backward()
            alices_opt.step()
            
        for data in harrys_loader:
            harrys_model.zero_grad()
            predict_value = harrys_model(data, device).to(torch.device("cpu"))  # [0, 1] -> recover
            loss = criterion(predict_value, data["flow_y"])
            epoch_loss_harry += loss.item()
            loss.backward()
            harrys_opt.step()
            
        for data in jacks_loader:
            jacks_model.zero_grad()
            predict_value = jacks_model(data, device).to(torch.device("cpu"))  # [0, 1] -> recover
            loss = criterion(predict_value, data["flow_y"])
            epoch_loss_jack += loss.item()
            loss.backward()
            jacks_opt.step()    
            
        for data in miks_loader:
            miks_model.zero_grad()
            predict_value = miks_model(data, device).to(torch.device("cpu"))  # [0, 1] -> recover
            loss = criterion(predict_value, data["flow_y"])
            epoch_loss_mik += loss.item()
            loss.backward()
            miks_opt.step()
            
        for data in olivers_loader:
            olivers_model.zero_grad()
            predict_value = olivers_model(data, device).to(torch.device("cpu"))  # [0, 1] -> recover
            loss = criterion(predict_value, data["flow_y"])
            epoch_loss_oliver += loss.item()
            loss.backward()
            olivers_opt.step()  
            
        for data in kats_loader:
            kats_model.zero_grad()
            predict_value = kats_model(data, device).to(torch.device("cpu"))  # [0, 1] -> recover
            loss = criterion(predict_value, data["flow_y"])
            epoch_loss_kat += loss.item()
            loss.backward()
            kats_opt.step()  
         
        for data in elizas_loader:
            elizas_model.zero_grad()
            predict_value = elizas_model(data, device).to(torch.device("cpu"))  # [0, 1] -> recover
            loss = criterion(predict_value, data["flow_y"])
            epoch_loss_eliza += loss.item()
            loss.backward()
            elizas_opt.step()  
            
        end_time = time.time()

        print("Epoch: {:04d}, Loss Bob: {:02.4f}, Loss Alice: {:02.4f}, Loss Harry: {:02.4f}, Loss Jack: {:02.4f}, Loss Mik: {:02.4f}, Loss Oliver: {:02.4f}, Loss Kat: {:02.4f}, Loss Eliza: {:02.4f}, Time: {:02.2f} mins".format(epoch, 1000 * epoch_loss_bob / len(train_data), 1000 * epoch_loss_alice / len(train_data), 1000 * epoch_loss_harry / len(train_data), 1000 * epoch_loss_jack / len(train_data), 1000 * epoch_loss_mik / len(train_data), 1000 * epoch_loss_oliver / len(train_data), 1000 * epoch_loss_kat / len(train_data), 1000 * epoch_loss_eliza / len(train_data),
                                                                          (end_time-start_time)/60))

In [3]:
if __name__ == '__main__':
    main()

Epoch: 0000, Loss Bob: 0.0092, Loss Alice: 0.0117, Loss Harry: 0.0125, Loss Jack: 0.2817, Loss Mik: 0.0089, Loss Oliver: 0.0114, Loss Kat: 1.4574, Loss Eliza: 0.0213, Time: 12.88 mins
Epoch: 0001, Loss Bob: 0.0092, Loss Alice: 0.0117, Loss Harry: 0.0125, Loss Jack: 0.2781, Loss Mik: 0.0089, Loss Oliver: 0.0113, Loss Kat: 1.4467, Loss Eliza: 0.0208, Time: 12.98 mins
Epoch: 0002, Loss Bob: 0.0092, Loss Alice: 0.0117, Loss Harry: 0.0125, Loss Jack: 0.2745, Loss Mik: 0.0089, Loss Oliver: 0.0113, Loss Kat: 1.4359, Loss Eliza: 0.0203, Time: 11.73 mins
Epoch: 0003, Loss Bob: 0.0092, Loss Alice: 0.0117, Loss Harry: 0.0125, Loss Jack: 0.2710, Loss Mik: 0.0089, Loss Oliver: 0.0113, Loss Kat: 1.4253, Loss Eliza: 0.0198, Time: 11.00 mins
Epoch: 0004, Loss Bob: 0.0092, Loss Alice: 0.0117, Loss Harry: 0.0125, Loss Jack: 0.2675, Loss Mik: 0.0089, Loss Oliver: 0.0112, Loss Kat: 1.4147, Loss Eliza: 0.0194, Time: 11.38 mins
Epoch: 0005, Loss Bob: 0.0092, Loss Alice: 0.0117, Loss Harry: 0.0125, Loss Jack